In [1]:
pip install pandas scikit-learn nltk


In [2]:
import pandas as pd


In [5]:

data = pd.read_csv('Combined.csv', encoding='latin1')



In [6]:
data.head()

,Name,Rating,total rating,Review
0,krishu k.,5/,5,it's very good product. I am purchasing again ...
1,Upender Malik,5/,5,Very nice product and very good brand... Smytt...
2,Faiziya Anam,5/,5,Make sure theyâ??re super smooth and soft and ...
3,Pragati Bhatia,4/,5,nice fragrance of the product can try it to se...
4,Tanya Sharma,5/,5,very good product in low budget must try guys ...


In [7]:
data.shape

(11575, 4)

In [8]:
data.describe()

,Name,Rating,total rating,Review
count,11575,11574,11574,11574
unique,1679,7,2,1738
top,Guest User,5/,5,This gel was awesome it was too smooth to appl...
freq,104,8744,11552,28


In [11]:
data.isnull().sum()

Name            0
Rating          0
total rating    0
Review          0
dtype: int64

In [10]:
data.dropna(inplace=True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11574 entries, 0 to 11573
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          11574 non-null  object
 1   Rating        11574 non-null  object
 2   total rating  11574 non-null  object
 3   Review        11574 non-null  object
dtypes: object(4)
memory usage: 452.1+ KB


In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [19]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [21]:
def preprocess_text(text):
    # Tokenize text
    words = word_tokenize(text)

    # Remove stop words and apply lemmatization
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha() and word.lower() not in stop_words]

    return " ".join(filtered_words)

data['clean_text'] = data['Review'].apply(preprocess_text)

In [22]:
from sklearn.model_selection import train_test_split


In [34]:
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['Rating'], test_size=0.2, random_state=42)


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11574 entries, 0 to 11573
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          11574 non-null  object
 1   Rating        11574 non-null  object
 2   total rating  11574 non-null  object
 3   Review        11574 non-null  object
 4   clean_text    11574 non-null  object
dtypes: object(5)
memory usage: 542.5+ KB


In [27]:
data.head()

,Name,Rating,total rating,Review,clean_text
0,krishu k.,5/,5,it's very good product. I am purchasing again ...,good product purchasing happy thnx smyttamn
1,Upender Malik,5/,5,Very nice product and very good brand... Smytt...,nice product good brand smytten help brand cus...
2,Faiziya Anam,5/,5,Make sure theyâ??re super smooth and soft and ...,make sure theyâ super smooth soft street every...
3,Pragati Bhatia,4/,5,nice fragrance of the product can try it to se...,nice fragrance product try see help make change
4,Tanya Sharma,5/,5,very good product in low budget must try guys ...,good product low budget must try guy recommend...


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)


In [29]:
from sklearn.ensemble import RandomForestClassifier


In [30]:
model=RandomForestClassifier(n_estimators=500)

In [32]:
X=data['clean_text']
y=data['Rating']

In [38]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [39]:
model.fit(X_train_tfidf, y_train)


RandomForestClassifier(n_estimators=500)

In [40]:
y_pred = model.predict(X_test_tfidf)


In [41]:
y_pred

array([' 5/', ' 5/', ' 5/', ..., ' 5/', ' 5/', ' 5/'], dtype=object)

In [43]:
from sklearn.metrics import accuracy_score,classification_report


In [45]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:}')

Accuracy: 0.9900647948164147


In [47]:
def predict_rating(new_review):
    cleaned_review = preprocess_text(new_review)
    review_tfidf = tfidf_vectorizer.transform([cleaned_review])
    predicted_rating = model.predict(review_tfidf)
    return predicted_rating[0]



In [51]:
new_review = "it was a good shade and i liked the quality and quantity and size too. loved it."
predicted_rating = predict_rating(new_review)
print(f'Predicted Rating: {predicted_rating}')

Predicted Rating:  2/


In [52]:
from sklearn.svm import SVC

In [53]:
m=SVC()

In [54]:
m.fit(X_train_tfidf, y_train)


SVC()

In [55]:
yp = m.predict(X_test_tfidf)

In [56]:
yp

array([' 5/', ' 5/', ' 5/', ..., ' 5/', ' 5/', ' 5/'], dtype=object)

In [57]:
accuracy = accuracy_score(y_test, yp)
print(f'Accuracy: {accuracy:}')

Accuracy: 0.9727861771058315


In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
l=LogisticRegression()

In [60]:

l.fit(X_train_tfidf, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [62]:
yl = l.predict(X_test_tfidf)

In [63]:
yl

array([' 5/', ' 5/', ' 5/', ..., ' 5/', ' 5/', ' 5/'], dtype=object)

In [64]:
accuracy = accuracy_score(y_test, yl)
print(f'Accuracy: {accuracy:}')

Accuracy: 0.9326133909287257
